In [1]:
import os
import json
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from tqdm import tqdm, trange
from shapely.geometry import shape, Point

In [2]:
cwd = os.getcwd()
folder = os.path.join(cwd, '../final_clean_version_2') # folder to store sales data from 520 suburbs
files = os.listdir(folder)
len(files)

520

In [3]:
# inspect the first dataset

pd.read_csv(os.path.join(folder, files[0])).head(1).transpose()

,0
PropID,Abbotsford_VIC_3067_0
Street,7/29 Church Street
PropStreet,7/29 Church Street
Suburb,Abbotsford
PropSuburb,Abbotsford
Region,VIC
PostalCode,3067
Latitude,-37.809156
Longitude,145.000665
Type,APARTMENT


# Merge into one single dataset

In [4]:
# keep only data from 2013-2017 for SOLD HOUSE that have prices recorded; merge into one single dataset

b4, after = [], []
for i, file in enumerate(tqdm(files)):
    
    if i==0:
        data_all = pd.read_csv(os.path.join(folder, file))
        # print(f"b4: {len(data_all)}")
        b4.append(len(data_all))
        data_all = data_all.dropna(subset=['Price']).loc[
            (data_all['SoldOrRent'] == 'SOLD') & (data_all['Type'] == 'HOUSE')
        ]
        after.append(len(data_all))
        # print(f"after: {len(data_all)}")
    else:
        newdata = pd.read_csv(os.path.join(folder, file))
        # print(f"b4: {len(newdata)}")
        b4.append(len(newdata))
        newdata = newdata.dropna(subset=['Price']).loc[
            (newdata['SoldOrRent'] == 'SOLD') & (newdata['Type'] == 'HOUSE')
        ]
        # print(f"after: {len(newdata)}")
        after.append(len(newdata))
        data_all = pd.concat([data_all, newdata])

100%|██████████| 520/520 [00:42<00:00, 12.21it/s]


In [5]:
data_all.shape

(1255109, 24)

# First clean

1. remove redundant columns
2. remove instances having LandSize and BuildingArea both missing
3. remove instances having Bedrms, Bathrms, Cars all missing
4. remove price that is below 100,000

In [6]:
data_clean = data_all.drop(
    columns=['Street', 'PropStreet', 'PropSuburb', 'Region', 'Type', 'SoldOrRent', 'Source']
).dropna(
    subset=['LandSize', 'BuildingArea'], how='all'
).dropna(
    subset=['Bedrms', 'Bathrms', 'Cars'], how='all'
)

data_clean.shape

(1183760, 17)

In [7]:
# this is raw data after simple cleaning, so it may contain some useless columns

# data_clean.reset_index().drop(columns='index').to_csv(os.path.join(cwd, 'data\\ver2_house_sold.csv'), index=False)

c:\ProgramData\Miniconda3\lib\site-packages\pandas\core\internals\blocks.py:2199: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [8]:
# get those features of interest

data_clean = pd.read_csv(os.path.join(cwd, 'data\\ver2_house_sold.csv'))
data = data_clean[[
    'PropID', 'Suburb', 'Bedrms', 'Bathrms', 'Cars', 'LandSize', 'BuildingArea', 'YearBuilt', 'Price', 'Day',
    'Month', 'Year'
]]

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1183760 entries, 0 to 1183759
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   PropID        1183760 non-null  object 
 1   Suburb        1183760 non-null  object 
 2   Bedrms        1182928 non-null  float64
 3   Bathrms       1183426 non-null  float64
 4   Cars          1142723 non-null  float64
 5   LandSize      1120415 non-null  float64
 6   BuildingArea  804082 non-null   float64
 7   YearBuilt     869875 non-null   float64
 8   Price         1183760 non-null  float64
 9   Day           356820 non-null   float64
 10  Month         1183595 non-null  float64
 11  Year          1183595 non-null  float64
dtypes: float64(10), object(2)
memory usage: 108.4+ MB


In [9]:
# num of entries with sold date
data.dropna(subset=['Day']).shape

(356820, 12)

In [17]:
# Houses sold more than once
data['PropID'].value_counts().count()-data['PropID'].value_counts().value_counts()[1]

311475

In [20]:
os.getcwd()

'c:\\Users\\YLfan\\OneDrive\\UniMelb\\Research\\OTL_housing_price'